In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
import seaborn as sns

In [7]:
ch = pd.read_csv('../not_on_git/mw_cat/chandra_filtered_sources.csv' , index_col = 'name')
ch_id = pd.read_csv('compiled_data_v3/id_frame.csv' , index_col='name')[['offset' , 'class']]
off = 1.01 
ch = pd.merge(ch_id[ch_id['offset']<off] , ch , left_index=True , right_index =True , how='right')
ch = ch.drop(columns = ['significance'  , 'offset' , 'ra' , 'dec' , 'var_flag' , 'var_inter_hard_flag'])
ch = ch.rename(columns = {
    'flux_aper_b' : 'b-csc' , 
    'flux_aper_h' : 'h-csc' ,
    'flux_aper_m': 'm-csc' ,
    'flux_aper_s': 's-csc' ,
    'flux_aper_u': 'u-csc' ,
})
ch['class'] = ch['class'].replace(np.nan , 'X')


,class,gal_l,gal_b,likelihood,b-csc,h-csc,m-csc,s-csc,u-csc,hard_hm,hard_hs,hard_ms,var_intra_index_b,var_intra_prob_b,ks_intra_prob_b,kp_intra_prob_b,var_inter_index_b,var_inter_prob_b,var_inter_sigma_b
name,,,,,,,,,,,,,,,,,,,
2CXO J003935.9-732725,X,304.096050,-43.644346,20.474157,0.000000e+00,7.141104e-15,1.529669e-15,0.000000e+00,0.000000e+00,0.231106,-0.058713,-0.307308,0.0,0.252877,0.726962,0.872329,NaN,NaN,NaN
2CXO J003936.7-731249,X,304.116127,-43.887309,42.244796,1.458552e-14,1.196830e-14,2.057132e-15,0.000000e+00,NaN,0.343535,-0.016240,-0.368520,0.0,0.474083,0.704944,0.324659,NaN,NaN,NaN
2CXO J004028.7-731106,X,304.031850,-43.919668,754.193576,1.094959e-13,8.183634e-14,1.744202e-14,1.216939e-14,NaN,0.269831,0.336040,0.064959,0.0,0.180151,0.708192,0.247447,NaN,NaN,NaN
2CXO J004506.3-730056,X,303.575841,-44.104667,279.529236,2.379848e-14,1.178519e-14,7.671868e-15,4.860946e-15,6.986833e-14,-0.106184,-0.066209,0.042473,0.0,0.154119,0.836030,0.810222,NaN,NaN,NaN
2CXO J004659.0-731918,X,303.374710,-43.802532,21.849369,0.000000e+00,0.000000e+00,1.259314e-15,6.172243e-16,8.098542e-14,0.151156,0.394753,0.218613,0.0,0.249450,0.709251,0.344393,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2CXO J220613.7-495727,X,345.582863,-51.190099,26.144839,3.452933e-14,2.859357e-14,7.190546e-15,0.000000e+00,NaN,-0.013741,0.224859,0.233604,0.0,0.178982,0.472714,0.611808,0.0,0.428810,3.002890e-06
2CXO J220614.6-500951,X,345.272648,-51.119708,42.761919,7.855447e-15,2.507461e-15,0.000000e+00,5.262109e-15,0.000000e+00,0.999375,-0.570893,-0.999375,1.0,0.592450,0.918676,0.787411,5.0,0.831532,4.978411e-06
2CXO J220618.4-500554,X,345.365084,-51.152644,70.283927,2.417635e-14,1.919246e-14,2.265201e-15,3.881337e-15,0.000000e+00,0.463460,0.056215,-0.417239,1.0,0.609084,0.919050,0.660959,0.0,0.002736,1.087005e-08


In [8]:
sdss = pd.read_csv('mw_cat/sdss.csv' , index_col='name')
sdss

,umag,gmag,rmag,imag,zmag
name,,,,,
2CXO J143005.9+311125,19.632,17.922,16.925,16.461,16.126
2CXO J143007.2+311442,21.320,20.172,19.208,18.872,18.427
2CXO J143009.6+311418,22.154,21.129,21.127,20.963,20.790
2CXO J143009.7+311257,16.425,21.997,22.101,24.072,20.722
2CXO J143011.3+311300,16.336,15.211,14.794,14.303,14.120
...,...,...,...,...,...
2CXO J073300.3+314105,23.230,23.121,25.175,25.016,23.458
2CXO J073306.8+314250,22.322,22.064,21.888,20.985,20.239
2CXO J073307.6+314420,21.368,21.411,21.275,21.247,21.536
